<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myProspects_Acc_Dis_Signals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=806805116f610a13374f322c075545596dc64b510d32098d2a4600c5ce2ead09
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas-ta


In [2]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

mypf = pd.read_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv')

mypf = mypf[(mypf['Criteria'] == 'V40') | (mypf['Criteria'] == 'V40N')]

#mypf = mypf[mypf['Nifty100'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()

Mounted at /content/drive


In [3]:
# star stocks & my stocks
star_stocks = list(mypf[mypf['StarStock'] == 1]['Stock'].values)
my_stocks = list(mypf[mypf['InPortfolio'] == 1]['Symbol'].values)

In [4]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

In [5]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365*5)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [6]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))
    stock_df = yfin.Ticker(stock_symbol).history(period='5y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg

    #print(stock_df)

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close'].rolling(window = short_window, min_periods = 1).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close'].rolling(window = mid_window, min_periods = 1).mean(),0)

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df['200_SMA_Lag10'] = stock_df['200_SMA'].shift(10)
    stock_df['Slope%_200'] = round((stock_df['200_SMA'] - stock_df['200_SMA_Lag10'])*100/stock_df['200_SMA'],2)
    stock_df['50_SMA_Lag5'] = stock_df['50_SMA'].shift(5)
    stock_df['Slope%_50'] = round((stock_df['50_SMA'] - stock_df['50_SMA_Lag5'])*100/stock_df['50_SMA'],2)

    #stock_df.ta.rsi(append=True)
    #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
    stock_df['Close'] = round(stock_df['Close'],0)
    #stock_df['RSI'] = round(stock_df['RSI'],0)
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df['Min'] = round(min(stock_df['Close']),0)
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Upside%ATH'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume','20_SMA','50_SMA','200_SMA','200_SMA_Lag10','50_SMA_Lag5'
                   ], axis=1, inplace=True)

    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:

    stock_id = stock_id.upper() + '.NS'
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Shares'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]
    tmp['Med_PE'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['MedianPE'].values[0]
    #tmp['Curr_PE'] = current_pe
    #tmp['BizOutlook'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['BizOutlook'].values[0]
    #tmp['BizDurability'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['BizDurability'].values[0]
    tmp['Conviction'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Conviction'].values[0]
    tmp['LatestQtr'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['LatestQtr'].values[0]
    tmp['StarStock'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['StarStock'].values[0]
    tmp['Criteria'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Criteria'].values[0]
    tmp['Strategy'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Strategy'].values[0]
    df_prec_dev = df_prec_dev.append(tmp)

df_prec_dev

,Date,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,Avg_Cost,Shares,Med_PE,Conviction,LatestQtr,StarStock,Criteria,Strategy
1233,2023-06-20 00:00:00+05:30,27680.0,3MINDIA.NS,19.55,1.16,2.02,29373.0,15332.0,12.06,6.0,NaN,NaN,NaN,M,1.0,1.0,V40N,NaN
1233,2023-06-20 00:00:00+05:30,356.0,5PAISA.NS,11.98,1.26,2.45,553.0,96.0,43.11,55.0,347.06,228.0,NaN,M,1.0,1.0,V40N,ATH
1233,2023-06-20 00:00:00+05:30,22997.0,ABBOTINDIA.NS,11.54,0.82,0.13,23094.0,6634.0,0.59,0.0,NaN,NaN,NaN,M,1.0,1.0,V40,NaN
1233,2023-06-20 00:00:00+05:30,2445.0,AKZOINDIA.NS,9.02,1.16,0.71,2508.0,1360.0,5.49,3.0,NaN,NaN,NaN,M,1.0,1.0,V40,NaN
672,2023-06-20 00:00:00+05:30,1595.0,ANGELONE.NS,19.92,0.98,3.04,1889.0,207.0,17.48,18.0,NaN,NaN,NaN,M,1.0,1.0,V40N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1233,2023-06-20 00:00:00+05:30,750.0,TTKPRESTIG.NS,-8.01,-1.10,0.42,1165.0,391.0,53.62,55.0,NaN,NaN,NaN,M,1.0,0.0,V40N,NaN
874,2023-06-20 00:00:00+05:30,38.0,UJJIVANSFB.NS,36.79,3.57,3.12,57.0,13.0,43.18,50.0,34.87,1350.0,16.8,M,1.0,1.0,V40N,ATH
1233,2023-06-20 00:00:00+05:30,1844.0,VINATIORGA.NS,-6.21,-0.92,0.16,2342.0,448.0,26.29,27.0,NaN,NaN,NaN,M,0.0,0.0,V40N,NaN
1233,2023-06-20 00:00:00+05:30,654.0,VIPIND.NS,-0.09,0.31,1.14,743.0,200.0,16.39,14.0,NaN,NaN,NaN,M,1.0,0.0,V40N,NaN


In [7]:
cols = ['Date', 'Close', 'Stock', 'Dev%_200', 'StarStock','Criteria']
df_prec_dev = df_prec_dev[cols]

In [8]:
df_acc_dis = pd.DataFrame()

for stock_id in stock_ids:
      stock_id = stock_id.upper() + '.NS'
      tmp = stock_prec_dev(stock_id)
      list1 = [stock_id, round(tmp[tmp['Dev%_200']<0]['Dev%_200'].mean(),0), round(tmp[tmp['Dev%_200']>0]['Dev%_200'].mean(),0)]
      tmp1 = pd.DataFrame(list1).T
      tmp1.columns = ['Stock', 'Acc', 'Dis']
      df_acc_dis = df_acc_dis.append(tmp1)

df_acc_dis

,Stock,Acc,Dis
0,3MINDIA.NS,-8.0,8.0
0,5PAISA.NS,-14.0,25.0
0,ABBOTINDIA.NS,-5.0,15.0
0,AKZOINDIA.NS,-6.0,8.0
0,ANGELONE.NS,-8.0,42.0
...,...,...,...
0,TTKPRESTIG.NS,-9.0,14.0
0,UJJIVANSFB.NS,-19.0,22.0
0,VINATIORGA.NS,-6.0,16.0
0,VIPIND.NS,-13.0,16.0


In [9]:
df_recomm = pd.merge(df_prec_dev, df_acc_dis)
df_recomm

,Date,Close,Stock,Dev%_200,StarStock,Criteria,Acc,Dis
0,2023-06-20 00:00:00+05:30,27680.0,3MINDIA.NS,19.55,1.0,V40N,-8.0,8.0
1,2023-06-20 00:00:00+05:30,356.0,5PAISA.NS,11.98,1.0,V40N,-14.0,25.0
2,2023-06-20 00:00:00+05:30,22997.0,ABBOTINDIA.NS,11.54,1.0,V40,-5.0,15.0
3,2023-06-20 00:00:00+05:30,2445.0,AKZOINDIA.NS,9.02,1.0,V40,-6.0,8.0
4,2023-06-20 00:00:00+05:30,1595.0,ANGELONE.NS,19.92,1.0,V40N,-8.0,42.0
...,...,...,...,...,...,...,...,...
72,2023-06-20 00:00:00+05:30,750.0,TTKPRESTIG.NS,-8.01,0.0,V40N,-9.0,14.0
73,2023-06-20 00:00:00+05:30,38.0,UJJIVANSFB.NS,36.79,1.0,V40N,-19.0,22.0
74,2023-06-20 00:00:00+05:30,1844.0,VINATIORGA.NS,-6.21,0.0,V40N,-6.0,16.0
75,2023-06-20 00:00:00+05:30,654.0,VIPIND.NS,-0.09,0.0,V40N,-13.0,16.0


In [10]:
# Accumulate
df_recomm[df_recomm['Dev%_200'] < df_recomm['Acc']]

,Date,Close,Stock,Dev%_200,StarStock,Criteria,Acc,Dis
60,2023-06-20 00:00:00+05:30,564.0,RAJESHEXPO.NS,-15.65,1.0,V40N,-12.0,12.0
74,2023-06-20 00:00:00+05:30,1844.0,VINATIORGA.NS,-6.21,0.0,V40N,-6.0,16.0


In [11]:
# Disburse
df_recomm[df_recomm['Dev%_200'] > df_recomm['Dis']]

,Date,Close,Stock,Dev%_200,StarStock,Criteria,Acc,Dis
0,2023-06-20 00:00:00+05:30,27680.0,3MINDIA.NS,19.55,1.0,V40N,-8.0,8.0
3,2023-06-20 00:00:00+05:30,2445.0,AKZOINDIA.NS,9.02,1.0,V40,-6.0,8.0
8,2023-06-20 00:00:00+05:30,4662.0,BAJAJ-AUTO.NS,18.93,1.0,V40,-6.0,9.0
17,2023-06-20 00:00:00+05:30,8036.0,CERA.NS,34.93,1.0,V40N,-7.0,18.0
22,2023-06-20 00:00:00+05:30,89.0,EQUITASBNK.NS,43.23,1.0,V40N,-13.0,21.0
24,2023-06-20 00:00:00+05:30,839.0,FINCABLES.NS,30.02,0.0,V40N,-13.0,24.0
29,2023-06-20 00:00:00+05:30,1169.0,HCLTECH.NS,12.15,1.0,V40,-8.0,12.0
32,2023-06-20 00:00:00+05:30,644.0,HDFCLIFE.NS,18.11,0.0,V40,-9.0,12.0
36,2023-06-20 00:00:00+05:30,1292.0,ICICIGI.NS,12.17,1.0,V40,-7.0,10.0
37,2023-06-20 00:00:00+05:30,563.0,ICICIPRULI.NS,19.59,0.0,V40,-10.0,15.0


In [12]:
stock_id = 'BAJFINANCE.NS'
df_recomm[df_recomm['Stock'] == stock_id]

,Date,Close,Stock,Dev%_200,StarStock,Criteria,Acc,Dis
11,2023-06-20 00:00:00+05:30,7249.0,BAJFINANCE.NS,10.0,1.0,V40,-14.0,18.0


In [15]:
df_recomm['Var'] = df_recomm['Dis'] - df_recomm['Acc']
df_recomm.sort_values(by = 'Var', ascending=False)

,Date,Close,Stock,Dev%_200,StarStock,Criteria,Acc,Dis,Var
68,2023-06-20 00:00:00+05:30,7770.0,TATAELXSI.NS,10.77,0.0,V40N,-16.0,38.0,54.0
20,2023-06-20 00:00:00+05:30,4539.0,DIXON.NS,23.94,1.0,V40N,-12.0,38.0,50.0
4,2023-06-20 00:00:00+05:30,1595.0,ANGELONE.NS,19.92,1.0,V40N,-8.0,42.0,50.0
46,2023-06-20 00:00:00+05:30,1560.0,LUXIND.NS,0.89,0.0,V40N,-19.0,29.0,48.0
73,2023-06-20 00:00:00+05:30,38.0,UJJIVANSFB.NS,36.79,1.0,V40N,-19.0,22.0,41.0
...,...,...,...,...,...,...,...,...,...
56,2023-06-20 00:00:00+05:30,13827.0,PGHH.NS,-0.36,0.0,V40,-4.0,8.0,12.0
26,2023-06-20 00:00:00+05:30,4547.0,GILLETTE.NS,-5.65,0.0,V40,-7.0,5.0,12.0
19,2023-06-20 00:00:00+05:30,572.0,DABUR.NS,4.21,1.0,V40,-5.0,7.0,12.0
18,2023-06-20 00:00:00+05:30,1644.0,COLPAL.NS,6.77,1.0,V40,-4.0,8.0,12.0
